This script shows how you can check goodness of fit.

First, you will need to import a few modules.

In [12]:
import os
import numpy as np
import pandas as pd
from scipy import stats
from fitter import Fitter
from fitter import get_distributions
from fitter import get_common_distributions
from statsmodels.stats.diagnostic import anderson_statistic as adtest

from scipy.stats import genextreme
from scipy.stats import genlogistic
from scipy.stats import pearson3

import statistics
import math

Read the data into a pandas dataframe

In [13]:
data = pd.read_csv('data/Flow.csv')


list dataframe columns and list of distributions to assess goodness of fit for

In [14]:
col_names = data.columns
list_of_dists = ["genextreme", "genlogistic", "pearson3"]


Then check statistics for each distribution

In [15]:
 
result_ks = []
result_ad = []   
for j in list_of_dists:
    dist = getattr(stats, j)
    param = dist.fit(data.loc[:,col_names[1]])
    a = stats.kstest(data.loc[:,col_names[1]],j, args=param) # KS test
    c = adtest(x=data.loc[:,col_names[1]],dist=dist,fit=False,params=param) # Anderson Darling test
    result_ks.append((j,a[0]))
    result_ad.append((j,c))

now generating  critical values for each distribution function

for GEV (this is to calculate the critical value which you have to compare it with to accept or reject the null hypothesis)

Usually 100,000 of iteration is required but for this case let's assume 1000 iteratio are sufficient

In [16]:

n_mc= 1000
KS_gev=[]
AD_gev=[]
n= len(data.loc[:,col_names[1]])
for i in range(0,n_mc):
    a= -0.1
    x_1= genextreme.rvs(a,size=n)
    dist = getattr(stats, "genextreme")
    param = dist.fit(x_1)
    a= stats.kstest(x_1, "genextreme",args=param)
    Ks_v= a[0]
    KS_gev.append(Ks_v)
    b= adtest(x=x_1,dist=genextreme,fit=False,params=param)
    AD_gev.append(b)
    
critical_v95_gev_ks = np.percentile(KS_gev,95)
critical_v95_gev_ad = np.percentile(AD_gev,95)


Repeat for Genlog

In [17]:

n_mc= 1000
KS_genlog=[]
AD_genlog=[]
n=len(data.loc[:,col_names[1]])
for i in range(0,n_mc):
    a= 0.1
    x_1= genlogistic.rvs(a,size=n)
    dist = getattr(stats, "genlogistic")
    param = dist.fit(x_1)
    a= stats.kstest(x_1, "genlogistic",args=param)
    Ks_v= a[0]
    KS_genlog.append(Ks_v)
    b= adtest(x_1,dist=genlogistic,fit=False,params=param)
    AD_genlog.append(b)
    
critical_v95_glo_ks = np.percentile(KS_genlog,95)
critical_v95_glo_ad = np.percentile(AD_genlog,95)

Repeat for or pe3

In [18]:

n_mc= 1000
KS_pe3=[]
AD_pe3=[]
n=len(data.loc[:,col_names[1]])
for i in range(0,n_mc):
    skew=0.1
    x_1= pearson3.rvs(skew,size=n)
    dist = getattr(stats, "pearson3")
    param = dist.fit(x_1)
    a= stats.kstest(x_1, "pearson3",args=param)
    Ks_v= a[0]
    KS_pe3.append(Ks_v)
    b= adtest(x_1,dist=pearson3,fit=False,params=param)
    AD_pe3.append(b)
    
critical_v95_pe3_ks = np.percentile(KS_pe3,95)
critical_v95_pe3_ad = np.percentile(AD_pe3,95)

The critical values are also available

If the user want to check using Kolmogorov–Smirnov goodness of fit test.

In [19]:

method = "KS"

if method == "KS":
        if result_ks[0][1] < critical_v95_gev_ks:
            print ("The data fits %s distribution"%(list_of_dists[0]))
        else:
            print ("The data does not fit %s distribution"%(list_of_dists[0]))
            
        if result_ks[1][1] < critical_v95_glo_ks:
            print ("The data fits %s distribution"%(list_of_dists[1]))
        else:
            print ("The data does not fit %s distribution"%(list_of_dists[1]))
            
        if result_ks[2][1] < critical_v95_pe3_ks:
            print ("The data fits %s distribution"%(list_of_dists[2]))
        else:
            print ("The data does not fit %s distribution"%(list_of_dists[2]))
          
    

The data fits genextreme distribution
The data fits genlogistic distribution
The data does not fit pearson3 distribution


If the user want to check using Anderson darling goodness of fit test

In [20]:
method = "AD"

if method == "AD":
        if result_ks[0][1] < critical_v95_gev_ad:
            print ("The data fits %s distribution"%(list_of_dists[0]))
        else:
            print ("The data does not fit %s distribution"%(list_of_dists[0]))
            
        if result_ks[1][1] < critical_v95_glo_ad:
            print ("The data fits %s distribution"%(list_of_dists[1]))
        else:
            print ("The data does not fit %s distribution"%(list_of_dists[1]))
            
        if result_ks[2][1] < critical_v95_pe3_ad:
            print ("The data fits %s distribution"%(list_of_dists[2]))
        else:
            print ("The data does not fit %s distribution"%(list_of_dists[2]))
    

The data fits genextreme distribution
The data fits genlogistic distribution
The data fits pearson3 distribution
